## Outside of the Container:

In [2]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import os
import shutil

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
def cropped_pc_from_bounds(pts: np.array, min_bounds: np.array, max_bounds: np.array):
    """
    Bounds a point cloud to a specified world boundary.

    Args:
      pts: A numpy array of shape (N, 3) representing the point cloud.
      min_bounds: A numpy array of shape (1, 3) representing the world bounds [x, y, z]
      max_bounds: A numpy array of shape (1, 3) representing the world bounds [x, y, z]

    Returns:
      A numpy array of shape (M, 3) representing the bounded point cloud,
      or None if no points remain within the bounds.
    """

    # Apply boolean mask to filter points within bounds
    valid = (
        (pts[:, 0] >= min_bounds[0])
        & (pts[:, 0] <= max_bounds[0])
        & (pts[:, 1] >= min_bounds[1])
        & (pts[:, 1] <= max_bounds[1])
        & (pts[:, 2] >= min_bounds[2])
        & (pts[:, 2] <= max_bounds[2])
    )
    pts_filtered = pts[valid]

    # Check if any points remain
    if not np.any(valid):
        print("no valid points")
        return None

    return pts_filtered


def crop_ply(file_key, bbox_dict, do_crop=True):
    if file_key not in bbox_dict:
        print("wrong file name")
        return
    file_path = bbox_dict[file_key]["path"]
    pcd = o3d.io.read_point_cloud(file_path)

    if do_crop:
        centroid = bbox_dict[file_key]["centroid"]
        dims = bbox_dict[file_key]["dims"]
        # Crop the point cloud
        min_bounds = [c - size / 2 for (c, size) in zip(centroid, dims)]
        max_bounds = [c + size / 2 for (c, size) in zip(centroid, dims)]

    else:
        centroid = pcd.get_center()
        min_bounds = pcd.get_min_bound()
        max_bounds = pcd.get_max_bound()

    filename = os.path.basename(file_path)
    name, _ = os.path.splitext(filename)

    directory = os.path.dirname(file_path)

    cropped_pc_path = os.path.join(directory, f"{name}.ply")
    cropped_pc_np_path = os.path.join(directory, f"{name}.npy")

    bbox_dict[file_key]["name"] = name
    bbox_dict[file_key]["np_path"] = cropped_pc_np_path
    bbox_dict[file_key]["cropped_path"] = cropped_pc_path

    xyz_min = np.minimum(min_bounds, max_bounds)
    xyz_max = np.maximum(min_bounds, max_bounds)

    cropped_point_cloud = pcd.crop(
        o3d.geometry.AxisAlignedBoundingBox(min_bound=xyz_min, max_bound=xyz_max)
    )

    # Save the cropped pc with color:
    o3d.io.write_point_cloud(cropped_pc_path, cropped_point_cloud)

    # Save just the points as a np file for
    print(np.asarray(cropped_point_cloud.points).shape)
    np.save(cropped_pc_np_path, np.asarray(cropped_point_cloud.points))

    print("Saved both np and ply files!")
    return bbox_dict


def pc_from_np_array(pc_confs):
    # Create PointCloud object
    point_cloud = o3d.geometry.PointCloud()
    points = np.zeros((len(pc_confs), 3))
    points[:, 0] = pc_confs["x"]
    points[:, 1] = pc_confs["y"]
    points[:, 2] = pc_confs["z"]
    point_cloud.points = o3d.utility.Vector3dVector(points)

    colors = np.zeros((len(pc_confs), 3))
    colors[:, 0] = pc_confs["r"]
    colors[:, 1] = pc_confs["g"]
    colors[:, 2] = pc_confs["b"]
    point_cloud.colors = o3d.utility.Vector3dVector(colors)

    return point_cloud

In [4]:
bbox_dict = {
    "real_kettlebell": {"path": "/home/mmicatka/Downloads/fused_ply_real.ply"},
    "sim_kettlebell": {"path": "/home/mmicatka/Downloads/fused_ply.ply"},
    "sim_kettlebell_2": {"path": "/home/mmicatka/Downloads/fused_ply_new.ply"},
}


bbox_dict = crop_ply("real_kettlebell", bbox_dict, do_crop=False)
bbox_dict = crop_ply("sim_kettlebell", bbox_dict, do_crop=False)
bbox_dict = crop_ply("sim_kettlebell_2", bbox_dict, do_crop=False)

# copy files into tsgrasp dir
pc_dir = "/home/mmicatka/Documents/raven_manipulation/src/grasp_synthesis/src/grasp_synthesis/tsgrasp/test_pointclouds"

for k, v in bbox_dict.items():
    name = v["name"]
    try:
        shutil.copy(v["np_path"], pc_dir)
    except Exception as ex:
        print(ex)

    try:
        # these doesn't exist yet, but create the entry:
        v["pc_confs_path_np"] = os.path.join(pc_dir, f"{name}_confs.npy")
        v["grasps_path"] = os.path.join(pc_dir, f"{name}_grasps.npy")
        v["pc_confs_path"] = os.path.join(pc_dir, f"{name}_confs.ply")

    except Exception as ex:
        print(ex)

(13964, 3)
Saved both np and ply files!
(20700, 3)
Saved both np and ply files!
(41615, 3)
Saved both np and ply files!


## In Container

```bash
(from inside /nodes)
./run.sh

# Inside container:
cd grasp_synth_ws/src/grasp_synthesis/src/grasp_synthesis/tsgrasp/
jupyter notebook --ip 0.0.0.0  --allow-root
```

In [18]:
# Standard Library
import numpy as np
import torch

assert torch.cuda.is_available(), "cant access cuda. yikes"
from predict_grasps import GraspPredictor
import yaml
import os


def model_metadata_from_yaml(yaml_file_path: str) -> dict:
    metadata = {}
    try:
        with open(yaml_file_path, "r", encoding="utf-8") as stream:
            metadata = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
    return metadata


# Create the predictor:
pkg_root = "/grasp_synth/grasp_synth_ws/src/grasp_synthesis/"
model_metadata = model_metadata_from_yaml(
    os.path.join(os.getcwd(), "models/tsgrasp_scene_1_frame/metadata.yaml")
)
model_path = os.path.join(pkg_root, model_metadata["ckpt_path"])
grasp_predictor = GraspPredictor(model_metadata, False, pkg_root)
print("done")

done


In [19]:
# Load the points:
pc_dir = os.path.join(os.getcwd(), "test_pointclouds")
files = os.listdir(pc_dir)
npy_files = [file for file in files if file.endswith(".npy")]

grasp_dict = {}
for file in npy_files:
    key = os.path.splitext(file)[0]

    # Load the points
    full_file_name = os.path.join(pc_dir, file)
    cropped_points = np.load(full_file_name)
    # Run the detector:
    (grasps_array, cm_array) = grasp_predictor.detect(cropped_points)
    print(f"working on {full_file_name}")
    print(cropped_points.shape)
    print(f"Grasp Results: {len(grasps_array)}")

    # save the points
    filename = os.path.join(pc_dir, f"{key}_confs.npy")
    np.save(filename, cm_array)

    # # Save the conf array:
    filename = os.path.join(pc_dir, f"{key}_grasps.npy")
    confs = grasps_array["confidences"]
    np.save(filename, confs)

working on /grasp_synth/grasp_synth_ws/src/grasp_synthesis/src/grasp_synthesis/tsgrasp/test_pointclouds/fused_ply_new.npy
(41615, 3)
Grasp Results: 20807
working on /grasp_synth/grasp_synth_ws/src/grasp_synthesis/src/grasp_synthesis/tsgrasp/test_pointclouds/fused_ply.npy
(20700, 3)
Grasp Results: 10350
working on /grasp_synth/grasp_synth_ws/src/grasp_synthesis/src/grasp_synthesis/tsgrasp/test_pointclouds/fused_ply_real.npy
(13964, 3)
Grasp Results: 6982


## Outside the Container


In [7]:
grasps = []
for k, v in bbox_dict.items():
    print(k)
    # load the pc_conf
    pc_confs_path = v["pc_confs_path_np"]
    pc_confs = np.load(pc_confs_path)
    print(pc_confs.shape)
    # pcd = pc_from_np_array(pc_confs)

    # # Write the pointcloud as a ply:
    # print(np.asarray(pcd.points).shape)
    # o3d.io.write_point_cloud(v["pc_confs_path"], pcd)

    # # load grasps
    # grasps_path = v["grasps_path"]
    # grasp_array = np.load(grasps_path)
    # grasp_array = grasp_array[grasp_array >= 0.1]

    # plt.style.use("fivethirtyeight")
    # plt.hist(grasp_array, bins=30, edgecolor="black")

    # # Add labels and title
    # plt.xlabel("Confidence [0.1 - 1]")
    # plt.ylabel("Frequency")
    # plt.title(f"{k} - Grasp Confidences")

    # # Show plot
    # plt.show()

real_kettlebell
(6982, 7)
sim_kettlebell
(10350, 7)
sim_kettlebell_2
(20807, 7)
